<a href="https://colab.research.google.com/github/eyaler/constrained/blob/main/pan3d/pan3d_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let our optima combine!

### A tutorial on combinatorial optimization with OR-Tools CP-SAT [[slides](https://docs.google.com/presentation/d/1XwGUkprVfCdDp9Z1ZMbcDOxTSLhVopPyiRTCGYWmDGQ/edit?usp=sharing), [video](https://www.youtube.com/watch?v=0gwp9ad2X4E)]

or

### Can any form (diamond, pyramid) contain all ~~26~~ 27 letters?

A

### [3D perfect pangram](https://eyalgruss.com/constrained/pan3d/?en) solution to a [Word Ways challenge](https://digitalcommons.butler.edu/cgi/viewcontent.cgi?article=2326&context=wordways) from 1979

by

### [Eyal Gruss](https://eyalgruss.com)

In [1]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

In [2]:
#@title The attack plan
from IPython.display import IFrame

IFrame(src='https://eyalgruss.com/constrained/pan3d/demo', width=1400, height=800)

In [3]:
#@title Install dependencies

!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!

In [4]:
#@title Download wordlist

!wget -nc https://github.com/eyaler/hebrew_wordlists/raw/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv -O wordlist.csv

--2024-11-04 07:55:42--  https://github.com/eyaler/hebrew_wordlists/raw/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/eyaler/hebrew_wordlists/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv [following]
--2024-11-04 07:55:43--  https://raw.githubusercontent.com/eyaler/hebrew_wordlists/refs/heads/main/intersect/mc4_intersect_no_fatverb.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3574389 (3.4M) [text/plain]
Saving to: ‘wordlist.csv’

wordlist.csv        100%[===================>]   3.41M  --.-KB/s    in 0.08s   

2024-11-04 07:

In [5]:
#@title Load and preprocess wordlist

with open('wordlist.csv', encoding='utf8') as f:
    wordlist = f.read().replace('ך', 'כ').replace('ם', 'מ').replace('ן', 'נ').replace('ף', 'פ').replace('ץ', 'צ').splitlines()

word2score = {w.split(',')[0]: int(w.split(',')[1]) for w in wordlist[::-1]}

for w, s in list(word2score.items())[-5:]:
  print(f'{w} {s:,}')

print()
for w, s in list(word2score.items())[:5]:
  print(f'{w} {s:,}')

עמ 23,385,649
לא 37,013,163
על 51,535,935
את 69,835,058
של 74,641,565

תתרשלנה 1
תתרקמו 1
תתרפטנה 1
תתרפטי 1
תתרנותי 1


In [6]:
#@title Filter wordlist for allowed words

limit = 2000 #@param {type: "integer"}

if not limit:
  limit = None

wordlist = [w for w in word2score if len(w) == 3
              and (len(set(w)) == 3 or len(set(w)) == 2 and any(w.count(c) == 2 for c in 'כמנפצ'))
              and '"' not in w and "'" not in w][::-1][:limit]
print(f'{len(wordlist)=}')

print()
for w in wordlist[:5]:
  print(f'{w} {word2score[w]:,}')

print()
for w in wordlist[-5:]:
  print(f'{w} {word2score[w]:,}')

len(wordlist)=2000

הוא 21,755,267
היא 12,740,845
אני 11,308,587
אבל 9,421,249
בינ 8,466,909

פוג 1,907
יאט 1,898
אגש 1,892
סכה 1,883
מחא 1,882


In [7]:
#@title Setup model and solver<br>(rerun from here if you modify constraints)

from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

# Uncomment if you want logging (see: https://d-krupke.github.io/cpsat-primer/05_parameters.html#logging):
#solver.parameters.log_search_progress = True

In [8]:
#@title Create cell variables for chars

cells = {}
flat = []
for x in range(3):
    for y in range(3):
        for z in range(3):
            cells[(x, y, z)] = model.new_int_var(ord('א'), ord('ת'), f'cells_{x}_{y}_{z}')
            flat.append(cells[(x, y, z)])

In [9]:
#@title Create boolean indicator variables,<br>channel them to the cell variables,<br>and add count constraints

is_char = {}
for c in 'אבגדהוזחטיכלמנסעפצקרשת':
    is_char[c] = []

    for i in range(27):
        is_char[c].append(model.new_bool_var(f'is_char_{c}_{i}'))

        # If-and-only-if condition (see: https://developers.google.com/optimization/cp/channeling):
        model.add(flat[i] == ord(c)).only_enforce_if(is_char[c][i])
        model.add(flat[i] != ord(c)).only_enforce_if(~is_char[c][i])

    if c in 'כמנפצ':
        model.add(sum(is_char[c]) == 2)
    else:
        model.add_exactly_one(is_char[c])

In [10]:
#@title Add sweetheart name constraint (optional)<br>also helps break solution symmetry, and runs faster

#@markdown Letters must be distinct, except כמנפצ which may appear twice:
char1 = 'י' # @param ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'כ', 'ל', 'מ', 'נ', 'ס', 'ע', 'פ', 'צ', 'ק', 'ר', 'ש', 'ת']
char2 = 'ע' # @param ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'כ', 'ל', 'מ', 'נ', 'ס', 'ע', 'פ', 'צ', 'ק', 'ר', 'ש', 'ת']
char3 = 'ל' # @param ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'כ', 'ל', 'מ', 'נ', 'ס', 'ע', 'פ', 'צ', 'ק', 'ר', 'ש', 'ת']
#@markdown Note: If you later change this, make sure to rerun from the model setup

name = char1 + char2 + char3
assert len(set(name)) == 3 or len(set(name)) == 2 and any(name.count(c) == 2 for c in 'כמנפצ'), 'Letters must be distinct, except כמנפצ which may appear twice'

if name not in wordlist:
  word2score[name] = word2score[wordlist[0]] + 1
  wordlist.insert(0, name)

model.add(cells[(0, 1, 1)] == ord(char1))
model.add(cells[(1, 1, 1)] == ord(char2))
model.add(cells[(2, 1, 1)] == ord(char3))

In [11]:
#@title Add allowed assignment constraints (words + scores)

allowed = [[ord(c) for c in w] + [word2score[w]] for w in wordlist]
for a in allowed[:5]:
  print(a)

score_values = [word2score[w] for w in wordlist]
scores = []


def new_score_var():
    scores.append(model.new_int_var(min(score_values), max(score_values), f'scores_{len(scores)}'))
    return scores[-1]


for i in range(3):
    for j in range(3):
        model.add_allowed_assignments([cells[(0, i, j)], cells[(1, i, j)], cells[(2, i, j)], new_score_var()], allowed)
        model.add_allowed_assignments([cells[(i, 0, j)], cells[(i, 1, j)], cells[(i, 2, j)], new_score_var()], allowed)
        model.add_allowed_assignments([cells[(i, j, 0)], cells[(i, j, 1)], cells[(i, j, 2)], new_score_var()], allowed)

[1492, 1493, 1488, 21755267]
[1492, 1497, 1488, 12740845]
[1488, 1504, 1497, 11308587]
[1488, 1489, 1500, 9421249]
[1489, 1497, 1504, 8466909]


In [12]:
#@title Solve to check feasibility (optional)

solver.solve(model)
print(solver.status_name())

OPTIMAL


In [13]:
#@title Solve to find optimal solution score

class ObjectiveLogger(cp_model.CpSolverSolutionCallback):
    def on_solution_callback(self):
        print(f'{self.objective_value=} {self.best_objective_bound=}')

worst = model.new_int_var(min(score_values), max(score_values), 'worst')
model.add_min_equality(worst, scores)
model.maximize(worst)

solver.solve(model, ObjectiveLogger())

print([(w, i) for i, w in enumerate(wordlist) if word2score[w] == solver.objective_value])

self.objective_value=1967.0 self.best_objective_bound=229540.0
self.objective_value=2565.0 self.best_objective_bound=229540.0
self.objective_value=3269.0 self.best_objective_bound=229540.0
[('צבנ', 1815)]


In [14]:
#@title Solve to find all optimal solutions<br>(having the optimal score found above)

class SolutionCollector(cp_model.CpSolverSolutionCallback):
    def __init__(self):
        super().__init__()
        self.solutions = []

    def decode(self, *x):
        return ''.join(chr(self.value(v)) for v in x)

    def on_solution_callback(self):
        solution = []
        for i in range(3):
            for j in range(3):
                solution.append(self.decode(cells[(0, i, j)], cells[(1, i, j)], cells[(2, i, j)]))
                solution.append(self.decode(cells[(i, 0, j)], cells[(i, 1, j)], cells[(i, 2, j)]))
                solution.append(self.decode(cells[(i, j, 0)], cells[(i, j, 1)], cells[(i, j, 2)]))

        self.solutions.append(solution)
        print(len(self.solutions), solution)


model.clear_objective()  # Needed for reusing the model to find multiple solutions
model.add(worst == round(solver.objective_value))  # Constrain solutions to the found objective value. Comment out to get all (non-optimal) solutions

solver.parameters.enumerate_all_solutions = True  # Note this disables parallelism
solution_collector = SolutionCollector()
solver.solve(model, solution_collector)

1 ['שחק', 'שאג', 'שטנ', 'טרפ', 'טיס', 'איכ', 'נפצ', 'נכה', 'גסה', 'אמצ', 'חמד', 'חרפ', 'יעל', 'רעמ', 'מעז', 'כזב', 'פזו', 'דמו', 'גדת', 'קצת', 'קפצ', 'סמכ', 'פלכ', 'צלב', 'הונ', 'צבנ', 'תכנ']
2 ['שחק', 'שטנ', 'שאג', 'אמצ', 'איכ', 'טיס', 'גדת', 'גסה', 'נכה', 'טרפ', 'חרפ', 'חמד', 'יעל', 'מעז', 'רעמ', 'סמכ', 'דמו', 'פזו', 'נפצ', 'קפצ', 'קצת', 'כזב', 'צלב', 'פלכ', 'הונ', 'תכנ', 'צבנ']


4

In [15]:
#@title The solution

from IPython.display import IFrame, HTML

chars = ''.join(solution_collector.solutions[0][::3])[::-1].replace('כ', 'ך', 1).replace('מ', 'ם', 1).replace('נ', 'ן', 1).replace('פ', 'ף', 1).replace('צ', 'ץ', 1)[::-1]
url = f'https://eyalgruss.com/constrained/pan3d/demo#{chars}'
display(HTML(f'<a href="{url}">{url}</a>'))
IFrame(src=url, width=1400, height=800)